<a href="https://colab.research.google.com/github/bessoh2/IA_DEM_Roughness/blob/Hannah_branch/Rolling_STDev_func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rolling Standard Deviation Function

#### Step 1: Install the rasterio package, since we're working with spatial data. Also import important packages for working with spatial data.

In [3]:
!pip install rasterio
import rasterio as rio
import xarray as xr
import matplotlib.pyplot as plt

     |████████████████████████████████| 19.1MB 48.5MB/s 


#### Step 2: Define the filepath to your data.  
We are using a .tif file, that is a terrain elevation file that was derived from aerial lidar in the Upper Tuolumne Watershed within Yosemite National Park. this specific file is centered around Tuolumne Meadows, a very flat region within the watershed at around 8500 ft. elevation.

In [8]:
fn = '/content/drive/My Drive/Image_Analysis_Proj/TuolumneMeadowsSubset.tif'

In [9]:
fn

'/content/drive/My Drive/Image_Analysis_Proj/TuolumneMeadowsSubset.tif'

In [6]:
!pwd

/content


#### Step 3: Open your dataset using xarray. This will load in the data as an xarry DataArray

In [10]:
# This version worked: loading in data with xarray to use the rolling function that is built-in.

im = xr.open_rasterio(fn)

# The code below is from the xarray documentation. I think you would use it to build back the coordinates (lat, lon) of the image. I haven't played with this yet. 

# from affine import Affine
# transform = Affine.from_gdal(*da.attrs['transform'])
# nx, ny = da.sizes['x'], da.sizes['y']
# x, y = np.meshgrid(np.arange(nx)+0.5, np.arange(ny)+0.5) * transform

RasterioIOError: ignored

#### Step 4: If the image is very large, subset the data to a size that will work for testing out the function on. In this case, we will subset to a 512x512 array.

In [ ]:
im_subset = im[:,0:511,0:511]

#### Step 5: Define a function that will compute the standard deviation within a moving window or a certain radius

In [ ]:
# This function computes the rolling stadard deviation, taking as input an xarray DataArray

def rolling_stdev(im, size):                # im: a DataArray loaded in using xarray. size: define the size (dimension of one side) of your moving window
  """
  Short Summary
  --------------------
  This function calculates the rolling standard deviation within a window of
  specified dimensions.

  Extended Summary
  ------------------
  This function utilizes xarray's rolling window operation to first create a 
  Rolling object, and then to apply that Rolling object to the DataArray
  to calculate standard deviation within each (size, size) window. The output is
  a new DataArray with each pixel populated with the standard deviation of all 
  pixels within the window for which the pixel was at the center. This function
  also outputs a plot of the output file for quick and easy data visualization.

  Parameters
  ------------
  im:   DataArray
        Generally a .tif file that has been loaded in using xarray.open_rasterio()
  size 
        The dimension of the square rolling window to use. If one dimension is supplied, 
        it will be propagated to the proper number of dimensions. For example, a size
        set to 3 will generate a rolling window square of dimensions (3,3).

  Returns
  ------------
  tuple
        consisting of the new DataArray of standard deviation values
        and the figure and axes of the plot of this new DataArray
  
  Example
  -------------
  Implement by saving the output as a variable. 
  >>>output = rolling_stdev(im=im_subset, size=3)
  >>>output[0]
  xarray.DataArray

    band: 1y: 511x: 511

    nan nan nan nan nan ... 0.06020909 0.1668852 0.025760788 0.05106023
    Coordinates:
        band
        (band)
        int64
        1
        y
        (y)
        float64
        4.2e+06 4.2e+06 ... 4.198e+06
        x
        (x)
        float64
        2.272e+05 2.273e+05 ... 2.288e+05
    Attributes: (0)

  """
  
  r = im.rolling(y=size)                    # this creates the xarray Rolling object 
  im_stdev = r.reduce(np.std)               # use the Rolling object to calculate the standard deviation within each window. Could substitute other functions like mean or median. 
  
  fig, ax = plt.subplots(figsize=(10,10))   # Plot the results
  c=ax.imshow(im_stdev.squeeze())           # Use squeeze() to allow matplotlib to plot a 3D array (in this case the array is of size (1,512,512)). It basically squeezes the dimension of size 1. 
  ax.axis('off')
  plt.colorbar(c)
  return im_stdev, fig, ax

In [ ]:
# Just included this function in case people felt intimidated using xarray to load datasets. I did not do documentation for this.
# This function reads in the data and computes the rolling standard deviation

def complete_stdev(fn, size):               # fn: the path to the file you want to read in. In my case it is a .tif file. size: the dimension of the square window 
  im = xr.open_rasterio(fn)                 # open the file as a DataArray using xarray
  im_subset = im[:,0:512,0:512]             # Subset the image to make it more manageable to test out this function on.
  r = im_subset.rolling(y=size)             # this creates the xarray Rolling object
  im_stdev = r.reduce(np.std)               # use the Rolling object to calculate the standard deviation within each window. Could substitute other functions like mean or median.

  fig, ax = plt.subplots(figsize=(10,10))   # Plot the results
  c=ax.imshow(im_stdev.squeeze())           # Use squeeze() to allow matplotlib to plot a 3D array (in this case the array is of size (1,512,512)). It basically squeezes the dimension of size 1.
  ax.axis('off')
  plt.colorbar(c)
  return fig, ax, im_stdev

#### Step 6: Use the function.  
Input the loaded DataArray and define a size for the moving window. Here I have used size = 3, which will create a 3x3 window.

In [ ]:
stdev = rolling_stdev(im_subset, 3)

In [ ]:
# Let's look at what we just made:

im_stdev[2]